In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
import tensorflow as tf
import keras.models
from keras.layers import Conv2D , MaxPool2D , Flatten , Dense , Dropout
from PIL import Image
from keras.models import Model , Sequential
from keras.layers import GlobalAveragePooling2D , BatchNormalization ;
from keras.layers import Dense , SimpleRNN , GRU , LSTM , Embedding
from tensorflow.keras.utils import to_categorical

In [21]:
trip = pd.read_csv(r"datasets/Trip_advisor_review.csv" )

In [22]:
trip.shape
trip.iloc[: , [0,1]]
trip.head()

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


In [23]:
cols = ['Rating' , 'Review']
trip = trip[cols]

In [24]:
trip

,Rating,Review
0,4,nice hotel expensive parking got good deal sta...
1,2,ok nothing special charge diamond member hilto...
2,3,nice rooms not 4* experience hotel monaco seat...
3,5,"unique, great stay, wonderful time hotel monac..."
4,5,"great stay great stay, went seahawk game aweso..."
...,...,...
20486,5,"best kept secret 3rd time staying charm, not 5..."
20487,4,great location price view hotel great quick pl...
20488,2,"ok just looks nice modern outside, desk staff ..."
20489,1,hotel theft ruined vacation hotel opened sept ...


In [36]:
trip_x = trip.iloc[: , 1]
trip_y = trip.iloc[: , 0]
from sklearn.model_selection import train_test_split

In [38]:
x_train , x_test , y_train , y_test = train_test_split(trip_x , trip_y ,test_size=0.2)

In [39]:
print(x_train.shape)
print(y_train.shape)
print("-----------")
print(x_test.shape)
print(y_test.shape)

(16392,)
(16392,)
-----------
(4099,)
(4099,)


In [40]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# to_categorical is used for Label one hot encoding

In [42]:
y_train

array([[0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0.]], dtype=float32)

In [43]:
y_train.shape

(16392, 6)

In [44]:
max_num_words  = 10000  # how many words do we consider from entire CORPUS
seq_len        = 50     # how many tokens(words to consider in each document / message)
embedding_size = 100    # vector length of each word

In [45]:
from keras.preprocessing.text import Tokenizer   # used for tokenization
from keras.preprocessing.sequence import pad_sequences  # used to bring all samples to same length

In [46]:
tokenizer = Tokenizer(num_words = max_num_words)
tokenizer.fit_on_texts(trip.Review)
x_train = tokenizer.texts_to_sequences(x_train)  # will convert the text to sequences of IDs
x_train = pad_sequences(x_train , maxlen= seq_len)

x_test = tokenizer.texts_to_sequences(x_test)
x_test = pad_sequences(x_test , maxlen= seq_len)

In [49]:
model = Sequential()     # initialize the network
model.add(Embedding(input_dim = max_num_words , 
                   input_length = seq_len ,
                   output_dim = embedding_size ) )

#------------------

model.add(LSTM(10))
model.add(Dense(6 , activation='softmax'))
model.compile(optimizer='adam' , loss = 'categorical_crossentropy' , metrics=['accuracy'] )
model.fit(x_train , y_train , epochs = 7 , batch_size = 32 , validation_split=0.2)

Epoch 1/7
410/410 [==============================] - 18s 36ms/step - loss: 1.2542 - accuracy: 0.4750 - val_loss: 1.0866 - val_accuracy: 0.5364
Epoch 2/7
410/410 [==============================] - 15s 36ms/step - loss: 0.9391 - accuracy: 0.6108 - val_loss: 1.0230 - val_accuracy: 0.5541
Epoch 3/7
410/410 [==============================] - 15s 38ms/step - loss: 0.7629 - accuracy: 0.6993 - val_loss: 1.0851 - val_accuracy: 0.5471
Epoch 4/7
410/410 [==============================] - 19s 48ms/step - loss: 0.6294 - accuracy: 0.7692 - val_loss: 1.1693 - val_accuracy: 0.5364
Epoch 5/7
410/410 [==============================] - 20s 48ms/step - loss: 0.4980 - accuracy: 0.8335 - val_loss: 1.2642 - val_accuracy: 0.5310
Epoch 6/7
410/410 [==============================] - 19s 46ms/step - loss: 0.4035 - accuracy: 0.8690 - val_loss: 1.3481 - val_accuracy: 0.5294
Epoch 7/7
410/410 [==============================] - 21s 51ms/step - loss: 0.3222 - accuracy: 0.9041 - val_loss: 1.4520 - val_accuracy: 0.5343

In [48]:
pred = model.predict(x_test)
pred

array([[5.6421524e-04, 9.9943572e-01],
       [9.8882301e-04, 9.9901116e-01],
       [2.3265025e-03, 9.9767345e-01],
       ...,
       [9.8769343e-04, 9.9901223e-01],
       [9.9641430e-01, 3.5857095e-03],
       [4.3154694e-04, 9.9956840e-01]], dtype=float32)

In [49]:
from sklearn.metrics import confusion_matrix
tab = confusion_matrix(np.argmax(pred , 1) , np.argmax(y_test , 1) ) 
tab

array([[ 192,    2],
       [  10, 1152]], dtype=int64)

In [50]:
Acc = tab.diagonal().sum() *100 / tab.sum()
Acc

99.11504424778761